In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

print(df.shape)
df.head()

(1462, 9)


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [24]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical = list(df.dtypes[(df.dtypes == 'int64') | (df.dtypes == 'float64')].index)

all_features = categorical + numerical

df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [ ]:
# Question 1

# Split dataset

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score

results = []

for n in numerical:
    df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    # Extract target FIRST
    target = n
    y_train = df_train[target].values
    y_val = df_val[target].values
    y_test = df_test[target].values

    # Remove target from dataframes
    df_train = df_train.drop(columns=[target])
    df_val = df_val.drop(columns=[target])
    df_test = df_test.drop(columns=[target])

    # One-hot encode
    dv = DictVectorizer(sparse=False)

    train_dict = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # train model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # predict
    y_pred = model.predict_proba(X_val)[:, 1]

    # auc
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    auc(fpr, tpr)
    if auc < 0.5:
        -df_train[n]


/Users/hjama/ai-learning/homework/mlzoomcamp/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.        , 0.        , 0.        , 0.00769231, 0.00769231,
       0.02307692, 0.02307692, 0.03076923, 0.03076923, 0.03846154,
       0.03846154, 0.05384615, 0.05384615, 0.06153846, 0.06153846,
       0.06923077, 0.06923077, 0.07692308, 0.07692308, 0.08461538,
       0.08461538, 0.12307692, 0.12307692, 0.14615385, 0.14615385,
       0.15384615, 0.15384615, 0.16153846, 0.16153846, 0.16923077,
       0.16923077, 0.17692308, 0.17692308, 0.2       , 0.2       ,
       0.21538462, 0.21538462, 0.22307692, 0.22307692, 0.23846154,
       0.23846154, 0.26153846, 0.26153846, 0.33846154, 0.33846154,
       0.34615385, 0.34615385, 0.35384615, 0.35384615, 0.36153846,
       0.36153846, 0.36923077, 0.36923077, 0.37692308, 0.37692308,
       0.46153846, 0.46153846, 0.46923077, 0.46923077, 0.50769231,
       0.50769231, 0.61538462, 0.61538462, 0.65384615, 0.65384615,
       0.66923077, 0.66923077, 0.70769231, 0.70769231, 1.        ])